In [5]:
import requests
import json
import warnings
import time
import pandas as pd
warnings.filterwarnings("ignore")

In [2]:
aoi_set = []

In [3]:
url = 'https://www.amap.com/service/poiInfo'
payload = {
    'query_type': 'TQUERY',
    'pagesize': 20,
    'pagenum': 1,
    'qii': True,
    'cluster_state': 5,
    'need_utd': True,
    'utd_sceneid': 1000,
    'div': 'PC1000',
    'addr_poi_merge': True,
    'is_classify': True,
    'zoom': 18,
    'city': 340100,
    'geoobj': '117.277801|31.878613|117.280389|31.883552',
    'keywords': '丁香家园一期'
}

headers = {
    'Host': 'www.amap.com',
    'amapuuid':'bdfc589d-8c81-49da-99dd-06500b389b02',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Referer': 'https://www.amap.com',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'x-csrf-token': None,
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6,zh-TW;q=0.5'
}

cookies = {
    'l':'dBaBgHluqWhTnkiQBOfwRurza77O5COj5sPzaNbMiICPOD7d4jTNWZKSrmO9CnGVH6zko3zIKqC8BWLHNPl8Z1u2AC1hE_iindC..',
    'isg':'BN_f1FuZ_nPvZPpThpWQExdxbjVpRDPmTfooqXEp-w7VAOWCdxXwNn4SwtDbmAte',
    'CNZZDATA1255626299':'728144284-1575339666-|1575339666',
}

In [6]:
df = pd.read_csv('../data/合肥_20191126.csv')
names = list(np.unique(df['address'].values))

In [ ]:
while len(names) > 0:
    payload['keywords'] = names.pop(0)
    print(payload['keywords'])
    
    r = requests.get(url, params=payload, headers=headers, cookies=cookies)
    content = r.text

    o = json.loads(content)
    if o['status'] == '1':
        for poi in o['data']['poi_list']:
            lock = False
            for item in aoi_set:
                if item['name'] == poi['name']:
                    lock = True
                    break

            if lock:
                continue

            aoi = None
            for i in poi['domain_list']:
                if i['id'] == '1013' and 'value' in i.keys():
                    aoi = i['value']

            if aoi is not None:
                aoi_set.append({
                    'shape': aoi,
                    'name': poi['name'],
                    'lng':  poi['longitude'],
                    'lat': poi['latitude']
                })
                print({
                    'shape': aoi,
                    'name': poi['name'],
                    'lng':  poi['longitude'],
                    'lat': poi['latitude']
                })
    else:
        print('reflection')
        break
    
    time.sleep(5)

琥珀新天地西苑
广福花园A区
信达天御枫丹阁
信达天御琅诗郡
东祥府
东祥府
信达天御琅诗郡
信达天御枫丹阁
天下锦城康桥郡
{'shape': '117.257781,31.813223_117.257688,31.813123_117.257706,31.811965_117.2577,31.810894_117.25774,31.810819_117.25779,31.810789_117.260132,31.810743_117.260492,31.81073_117.260676,31.810856_117.260663,31.811517_117.260799,31.812084_117.260983,31.812877_117.260856,31.812905_117.26085,31.813038_117.260791,31.813146_117.257781,31.813223', 'name': '天下锦城康桥郡', 'lng': '117.259238', 'lat': '31.811996'}
高铁都市花园和苑
{'shape': '117.295872,31.80416_117.297863,31.804841_117.298234,31.804462_117.298468,31.804266_117.298389,31.804078_117.296421,31.803465_117.296232,31.803934_117.295771,31.803764_117.295282,31.80352_117.294653,31.803198_117.294205,31.803684_117.295553,31.804559_117.295872,31.80416', 'name': '瑞和苑', 'lng': '117.296404', 'lat': '31.804041'}
{'shape': '117.234324,31.804797_117.235234,31.804799_117.235264,31.804779_117.23629,31.80397_117.236512,31.803849_117.237712,31.803813_117.238728,31.803817_117.240181,31.80381_117.240257